In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import random
import os
from fake_useragent import UserAgent as ua
from random import choice

if not os.path.exists('./restList'):
    os.mkdir('./restList')
    
# if not os.path.exists('./restPhoto'):
#     os.mkdir('./restPhoto')

vvls=['18.235.220.172',
 '20.94.229.106',
 '35.172.135.29',
 '77.247.126.158',
 '143.198.239.216',
 '92.204.129.161',
 '162.243.167.58',
 '167.172.132.136',
 '132.145.137.26',
 '3.136.37.78',
 '205.185.118.53',
 '103.83.37.11',
 '52.25.159.130',
 '12.218.209.130',
 '24.172.82.94',
 '159.203.188.130',
 '52.251.88.114',
 '172.245.92.238',
 '165.227.80.176',
 '51.81.82.175',
 '165.227.71.60',
 '198.255.161.102',
 '157.230.95.4',
 '162.243.168.215',
 '216.236.198.30',
 '209.126.13.170',
 '50.16.33.219']

url='https://www.tripadvisor.com.tw/Restaurants-g293913-oa8250-Taipei.html'
#url='https://www.tripadvisor.com.tw/Restaurants-g293913-Taipei.html'

# userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36 Edg/90.0.818.56'
# headers = {'User-Agent':userAgent}

ss=requests.session()

t=1
for i in range(150):
    ua1=ua().random
    headers = {'User-Agent':"".format(ua1)}
    a_=choice(vvls)
    proxies1 = { 'http': 'http://' + a_ }
    print('********載入第'+str(t)+'頁********')
    res=None
    while True:
        try:
            res=ss.get(url, headers=headers, proxies=proxies1)
            break
        except:
            pass
    
    soup=BeautifulSoup(res.text, 'html.parser')
    for restTags in soup.select('a[class="_15_ydu6b"]'):
        if len(restTags.text.split('. ',1))==1: # 該餐廳為贊助商
            continue
        else:
            restName=restTags.text.split('. ',1)[1]
        try:
            restUrl='https://www.tripadvisor.com.tw'+restTags['href']
            restRes=None
            while True:
                try:
                    restRes=ss.get(restUrl, headers=headers, proxies=proxies1)
                    break
                except:
                    pass
                    
            restSoup=BeautifulSoup(restRes.text, 'html.parser')
            if len(restSoup.select('svg[class="zWXXYhVR"]'))!=0:
                genRating=restSoup.select('svg[class="zWXXYhVR"]')[0]['title'].split(' 分')[0]
            else:
                genRating="None"
            if len(restSoup.select('a[class="_2mn01bsa"]'))!=0 and '$' in restSoup.select('a[class="_2mn01bsa"]')[0].text:
                price=restSoup.select('a[class="_2mn01bsa"]')[0].text
            else:
                price="None"
            if len(restSoup.select('a[class="_15QfMZ2L"]'))!=0:
                ranking=restSoup.select('a[class="_15QfMZ2L"]')[0].text
            else:
                ranking="None"
            if len(restSoup.select('a[class="_15QfMZ2L"]'))>=1:
                location=restSoup.select('a[class="_15QfMZ2L"]')[1].text
            else:
                location="None"
            if len(restSoup.select('span[class="_15QfMZ2L"]'))!=0:
                tel=restSoup.select('span[class="_15QfMZ2L"]')[0].text
            else:
                tel="None"                        
#             website=restSoup.select('a[class="_2wKz--mA _15QfMZ2L"]')[0]['href']
#             imgUrl=restSoup.select('img[class="basicImg"]')[0]['data-lazyurl']
#             resImg = requests.get(imgUrl, headers=headers)
#             resContent = resImg.content
#             with open('./restPhoto/{}.{}'.format(restName.replace('/', '-'), imgUrl.split('.')[-1]), 'wb') as f:
#                 f.write(resContent)
            with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'w', encoding='utf-8') as f:
                f.write('餐廳名稱: '+restName+'\n')
                f.write('餐廳網站: '+restUrl+'\n')
                f.write('區域排名: '+ranking+'\n')
                f.write('餐廳地點: '+location+'\n')
                f.write('餐廳電話: '+tel+'\n')
                f.write('餐廳價位: '+price+'\n')
                f.write('整體評分: '+genRating+'\n')
                
            ratingsList=restSoup.select('div[class="jT_QMHn2"]')
            if len(ratingsList) == 4:
                for ratings in ratingsList:
                    bubbles=ratings.select('span[class="_377onWB-"] span')[0]['class'][1].split('bubble_')[-1]
                    with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                        f.write(ratings.text+': ')
                        f.write(str(int(bubbles)/10)+'\n')
            elif len(ratingsList) == 3:
                for ratings in ratingsList:
                    bubbles=ratings.select('span[class="_377onWB-"] span')[0]['class'][1].split('bubble_')[-1]
                    with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                        f.write(ratings.text+': ')
                        f.write(str(int(bubbles)/10)+'\n')
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('氛圍: '+'None'+'\n')
            else:
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('食物: '+'None'+'\n')
                    f.write('服務: '+'None'+'\n')
                    f.write('實惠: '+'None'+'\n')
                    f.write('氛圍: '+'None'+'\n')
                    
            if len(restSoup.select('a[class="_2mn01bsa"]'))!=0 and '$' not in restSoup.select('a[class="_2mn01bsa"]')[0].text:
                restTypeList=restSoup.select('a[class="_2mn01bsa"]')
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('餐廳類型: ')
                    for restType in restTypeList:
                        f.write(restType.text+'/')
                    f.write(' '+'\n')
            elif len(restSoup.select('a[class="_2mn01bsa"]'))!=0 and '$' in restSoup.select('a[class="_2mn01bsa"]')[0].text: 
                restTypeList=restSoup.select('a[class="_2mn01bsa"]')[1:]
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('餐廳類型: ')
                    for restType in restTypeList:
                        f.write(restType.text+'/')
                    f.write(' '+'\n')                
            else:
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('餐廳類型: '+'None'+'\n')
                    
            print(restName+': 輸出成功')
        
        except IndexError as e:
            print('-----'+restName+': 餐廳資訊問題-----')
            print(e)
        except OSError as e:
            print('-----餐廳命名問題-----')
            print(e)
            
        time.sleep(random.randint(1, 3))
        
    t+=1
    newUrl = 'https://www.tripadvisor.com.tw'+soup.select('a[class="nav next rndBtn ui_button primary taLnk"]')[0]['href']
    url=newUrl

********載入第1頁********
後驛鵝肉店: 輸出成功
一口壽司: 輸出成功
圍裙: 輸出成功
無聊咖啡: 輸出成功
麵屋牛一雞骨牛肉麵: 輸出成功
燒桶子 韓風立燒: 輸出成功
Go! Cafe 無國界風味料理: 輸出成功
台灣媳婦切仔麵: 輸出成功
飽飽食堂: 輸出成功
小品雅廚: 輸出成功
高三孝松菸文創店: 輸出成功
銀咖哩: 輸出成功
YOLO’s Cafe - Chang'an East Road: 輸出成功
為福樓(忠孝店): 輸出成功
一起吧: 輸出成功
張媽媽牛肉麵: 輸出成功
樂樂炭火: 輸出成功
天銅火鍋達人(龍江店): 輸出成功
布克湖義式坊: 輸出成功
溫州大餛飩: 輸出成功
大觀園: 輸出成功
八月堂: 輸出成功
煎餅磨坊: 輸出成功
永和四海豆漿大王 - 萬華康定店: 輸出成功
壺同燒肉3號店: 輸出成功
白木屋: 輸出成功
南港鐵麵線: 輸出成功
Subway 長安東路: 輸出成功
麥味登早餐（北市京站）: 輸出成功
Shell Out Taiwan 一號店: 輸出成功
********載入第2頁********
麥味登早餐（北市京站）: 輸出成功
Shell Out Taiwan 一號店: 輸出成功
鍋比盆大港式涮涮鍋: 輸出成功
志明牛肉拉麵: 輸出成功
豚人拉麵: 輸出成功
安安海鮮: 輸出成功
宥祐素食坊: 輸出成功
香港88港式茶餐廳: 輸出成功
Arrival Cafe & Bistro: 輸出成功
林家潁精緻麵店: 輸出成功
一品巧廚: 輸出成功
金泰刀削麵: 輸出成功
秋月日式炸豬排: 輸出成功
夯吐司: 輸出成功
美珍香- 雙連店: 輸出成功
北海道GARAKU湯咖哩: 輸出成功
調色盤餐廳: 輸出成功
桔梗燉飯: 輸出成功
晴光姑嫂麵: 輸出成功
摩斯漢堡 (羅斯福路店): 輸出成功
德增豆漿店: 輸出成功
Pin Sheng Delightful Veggie House: 輸出成功
陳記外省麵: 輸出成功
江太壽司: 輸出成功
台南阿輝炒鱔魚: 輸出成功
沛緹歐美式餐廳: 輸出成功
Kidsorrow: 輸出成功
大鬍子老爹麵店: 輸出成功
來來順餡餅粥: 輸出成功
弘爺漢堡 福港店: 輸出成功
********載入第3頁********
來來順餡餅粥: 輸出成功
弘爺漢堡 福港店: 輸出成功
肥鰻

原味錄黑糖刨冰 天母總店: 輸出成功
甘心咖啡: 輸出成功
Dayan Café達洋咖啡館: 輸出成功
翠雍庭: 輸出成功
歐華軒: 輸出成功
欣葉鐘菜: 輸出成功
六廚川客料理: 輸出成功
小周切仔麵: 輸出成功
小川鍋物: 輸出成功
六順豆花: 輸出成功
廚房客家小館: 輸出成功
敝所: 輸出成功
********載入第19頁********
敝所: 輸出成功
侯薑母鴨: 輸出成功
Crazy Cart Cafe: 輸出成功
Mark’s Shabu: 輸出成功
儂來魚翅: 輸出成功
哲思軒 台北福華大飯店: 輸出成功
T+T: 輸出成功
CoCo壱番屋 高島屋Kitchen店: 輸出成功
焱鐵板燒: 輸出成功
淺淾: 輸出成功
柏克金啤酒(南京): 輸出成功
永寶餐廳: 輸出成功
府城蘭香排骨飯 富陽店: 輸出成功
Lavazza: 輸出成功
橘村屋 - 北平西路: 輸出成功
川湘号 香辣館: 輸出成功
柬式廚房厝邊咖啡: 輸出成功
樂吉: 輸出成功
Peppino 培皮諾小館: 輸出成功
海世界活海鮮廳: 輸出成功
老爸客家菜主題餐廳: 輸出成功
龍記手工水餃: 輸出成功
鐵板王: 輸出成功
香積廚: 輸出成功
Chuoyinshi Landmark: 輸出成功
何記大腸麵線: 輸出成功
張吳記麵餅舖: 輸出成功
佳佳豆花: 輸出成功
若荷: 輸出成功
Dongyang Gongtai Canteen (Zhongxiao Branch): 輸出成功
********載入第20頁********
年三十: 輸出成功
谛・Veganism 共响空间: 輸出成功
金品川菜海鮮餐廳: 輸出成功
老彭的菜: 輸出成功
台灣牛牛肉麵: 輸出成功
馬叔餅鋪: 輸出成功
藍象廷泰式火鍋 - 師大店: 輸出成功
考乍熋寶: 輸出成功
饗家: 輸出成功
滷肉控: 輸出成功
慶泰大飯店-金穗坊西餐廳: 輸出成功
AL CICCHETTO義麵坊小酒館: 輸出成功
普魯士德國豬腳: 輸出成功
美璟越南美食: 輸出成功
青島包子: 輸出成功
金色漁家餐廳: 輸出成功
Roots Lodge Café(仁愛店): 輸出成功
The Bed: 輸出成功
海豐快炒100: 輸出成功
湯布院本川製麵所（微風台北車站）: 輸出成功
成吉思汗餐廳: 輸出成功
台北陶園經典飯店: 輸出成功
金廚北

In [ ]:
import pandas as pd
import glob

data = list()
for filePath in glob.glob('./restList/*.txt'):
    with open(filePath, 'r', encoding='utf-8') as f:
        tmpList = f.read().split('\n')
        data.append(tmpList)
columns = [r.split(': ') [0] for r in data[0]]

df = pd.DataFrame(data=data, columns=columns)

for c in df:
    df[c] = df[c].apply(lambda x: x.split(': ')[-1])

In [ ]:
df

In [ ]:
df.to_csv('./restList.csv', index=False, encoding='utf-8-sig')

In [ ]:
restSoup.select('a[class="_2wKz--mA _15QfMZ2L"]')[0] # 官方網站網址未出現在標籤中